In [2]:
# !pip install torch --quiet
# !pip uninstall torchtext -y
# !pip install torchtext==0.10.0 --quiet

In [3]:
from torchtext.datasets import IMDB as YelpReviewFull

help(YelpReviewFull)

Help on function IMDB in module torchtext.datasets.imdb:

IMDB(root='.data', split=('train', 'test'))
    IMDB dataset
    
    Separately returns the train/test split
    
    Number of lines per split:
        train: 25000
    
        test: 25000
    
    
    Number of classes
        2
    
    
    Args:
        root: Directory where the datasets are saved.
            Default: .data
        split: split or splits to be returned. Can be a string or tuple of strings.
            Default: ('train', 'test')



In [4]:
# !pip install torchdata  # restart after installation
train_iter = YelpReviewFull(split='train')

aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 39.6MB/s]


In [5]:
next(train_iter)

('neg',
 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [6]:
for (line_number, (label, line)) in enumerate(train_iter):
  print(label, line)
  if line_number == 19:
    break 

neg "I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn't matter what one's political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn't true. I've seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don't exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies like The Brown Bunny, in which we're treated to the site of Vincent Gallo's throbbing johnson, but not a trace of pink visible on Chloe Sevigny. Before crying (or implying) "double-standard" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on display when actresses appears nude, and the same 

In [7]:
from torch.utils.data import DataLoader
train_iter = YelpReviewFull(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False)

In [8]:
next(iter(dataloader))

[('neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg'),
 ('I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. 

In [9]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [10]:
tokenizer = get_tokenizer("basic_english")
train_iter = YelpReviewFull(split='train')

In [11]:
def yield_tokens(data_iter):
  for _, text in data_iter:
    yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])

In [12]:
vocab.set_default_index(vocab["<unk>"])

In [13]:
# vocab(['here', 'is', 'an', 'alien', 'invasion', 'bangalore'])
vocab["<unk>"]

0

In [15]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: 0 if x == 'neg' else 1

text_pipeline("I am sid")

[12, 245, 5046]

In [16]:
from torch.utils.data import DataLoader
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# def collate_fn(batch):
#   src_batch, tgt_batch = [], []
#   for src_batch, tgt_batch in batch:
#     src_batch.append(text_transform[SRC_language](src_sample.rstrip('\n')))
#     tgt_batch.append(text_transform[SRC_language](src_sample.rstrip('\n')))
#   src_batch = pad_sequences(src_batch, padding_value=PAD_IDX)
#   tgt_batch = pad_sequences(tgt_batch, padding_value=PAD_IDX)
#   return src_batch, tgt_batch

In [18]:
weight = torch.randn(3,4)
weight

tensor([[-0.2552, -0.6535,  0.7490, -0.2083],
        [-0.0400,  0.4390,  0.0616, -1.1454],
        [ 2.0609, -0.7354, -0.5719,  0.9019]])

In [19]:
indices = torch.tensor([2,1])
indices

tensor([2, 1])

In [20]:
embedding = torch.nn.functional.embedding(indices, weight)
embedding

tensor([[ 2.0609, -0.7354, -0.5719,  0.9019],
        [-0.0400,  0.4390,  0.0616, -1.1454]])

In [22]:
embedding.mean(dim=0, keepdim=True)

tensor([[ 1.0105, -0.1482, -0.2552, -0.1217]])

In [23]:
torch.nn.functional.embedding_bag(indices, weight, torch.tensor([0]), mode='mean')

tensor([[ 1.0105, -0.1482, -0.2552, -0.1217]])

In [ ]:
def collate_fn(batch):
  label_list, text_list, offsets = [], [], [0]
  for (_label, _text) in batch:
    label_list.append(label_pipeline(_label))
    processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
    text_list.append(processed_text)
    offsets.append(processed_text.size(0))